# Deep kernel learning for data reconstruction and automated experiment



## Installation

In [ ]:
!pip install -q gpax
!pip install -q atomai  # we will use the atomai's utility function for preparing the input data
!pip install sidpy
!pip install bglib
!pip install --upgrade gdown

## Imports

In [ ]:
from warnings import filterwarnings

import numpy as np
import matplotlib.pyplot as plt
import math
import os

from sklearn.model_selection import train_test_split

import gpax
from atomai.utils import get_coord_grid, extract_patches_and_spectra, extract_subimages

gpax.utils.enable_x64()
filterwarnings("ignore", module="haiku._src.data_structures")
import h5py
import sidpy
import pyUSID as usid
from BGlib import be as belib
# from celluloid import Camera

## Prepared data

Download training data:

In [ ]:
!gdown https://drive.google.com/uc?id=1D5P4pxGEyk_R09k6Iwx8pTCC-z5T2SfX

Load data into the notebook:

In [ ]:
input_file = "/content/BEPS_1d5um_PTO_0001.h5"
h5_f = h5py.File(input_file, 'r+')
sidpy.hdf.hdf_utils.print_tree(h5_f)

In [ ]:
sho_mat = h5_f['Measurement_000/Channel_000/Raw_Data-SHO_Fit_001/Fit']
spec_val = h5_f['Measurement_000/Channel_000/Raw_Data-SHO_Fit_000/Spectroscopic_Values']
pos_inds = h5_f['Measurement_000/Channel_000/Position_Indices']
pos_dim_sizes = [np.max(pos_inds[:,0])+1, np.max(pos_inds[:,1]+1)]
topo = h5_f['Measurement_000/Channel_001/Raw_Data/']['r']

sho_mat_ndim = sho_mat[:].reshape(pos_dim_sizes[0], pos_dim_sizes[1], -1)
amp_mat_ndim = sho_mat_ndim['Amplitude [V]']
phase_mat_ndim = sho_mat_ndim['Phase [rad]']
fre_mat_ndim = sho_mat_ndim['Frequency [Hz]']
q_mat_ndim = sho_mat_ndim['Quality Factor']

In [ ]:
plt.figure()
plt.hist(phase_mat_ndim.ravel(),bins = 100);

min_ = np.min(phase_mat_ndim)
max_ = np.max(phase_mat_ndim)

pha_correct = np.where(phase_mat_ndim > 1.5, phase_mat_ndim + (min_-max_), phase_mat_ndim)

pha_correct = pha_correct + np.pi-0.75

plt.figure()
plt.hist(pha_correct.reshape(-1),bins = 100);
plt.axvline(np.pi, color ='red')
plt.axvline(0, color ='red')

In [ ]:
#separate on field and off field
full_spec_len = sho_mat_ndim.shape[2]
spec_len = int(full_spec_len/2)
pix_x, pix_y = sho_mat_ndim.shape[0], sho_mat_ndim.shape[1]

amp_off_field = np.zeros((pix_x, pix_y, spec_len))
pha_off_field = np.zeros((pix_x, pix_y, spec_len))
fre_off_field = np.zeros((pix_x, pix_y, spec_len))

amp_on_field = np.zeros((pix_x, pix_y, spec_len))
pha_on_field = np.zeros((pix_x, pix_y, spec_len))
fre_on_field = np.zeros((pix_x, pix_y, spec_len))

v_step = np.zeros(spec_len)

for i in range (spec_len):
  amp_off_field[:,:, i] = amp_mat_ndim[:,:,2*i+1]
  pha_off_field[:,:, i] = pha_correct[:,:,2*i+1]
  fre_off_field[:,:, i] = fre_mat_ndim[:,:,2*i+1]/1000
  amp_on_field[:,:, i] = amp_mat_ndim[:,:,2*i]
  pha_on_field[:,:, i] = pha_correct[:,:,2*i]
  fre_on_field[:,:, i] = fre_mat_ndim[:,:,2*i]/1000
  v_step[i] = spec_val[0,2*i]

pola_off_field = amp_off_field*np.cos(pha_off_field)

In [ ]:
f, (ax1,ax2, ax3, ax4) = plt.subplots(1,4, figsize = (24,4))
ax1.plot(v_step, amp_off_field.mean(axis = (0, 1)))
ax1.set_xlabel('Voltage (V)')
ax1.set_ylabel('Amplitude (a.u.)')

ax2.plot(v_step, pha_off_field.mean(axis = (0, 1)))
ax2.set_xlabel('Voltage (V)')
ax2.set_ylabel('Phase (rad)')

ax3.plot(v_step, fre_off_field.mean(axis = (0, 1)))
ax3.set_xlabel('Voltage (V)')
ax3.set_ylabel('Frequency (kHz)')

ax4.plot(v_step, pola_off_field.mean(axis = (0, 1)))
ax4.set_xlabel('Voltage (V)')
ax4.set_ylabel('Polarization (a.u.)')

In [ ]:
plt.imshow(amp_off_field.mean(2))

In [ ]:
struc_img = amp_off_field.mean(2)
plot_pixx1 = 50; plot_pixy1 = 50
plot_pixx2 = 20; plot_pixy2 = 20
# Plot spectrum and image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))
ax1.plot(v_step, pola_off_field[plot_pixx1, plot_pixy1], c = 'k')
ax1.plot(v_step, pola_off_field[plot_pixx2, plot_pixy2], c = 'r')
ax1.set_xlabel('Voltage (V)')
ax1.set_ylabel('Polarization (a.u.)')

ax2.imshow(struc_img)
ax2.scatter(plot_pixx1, plot_pixy1, marker='x', s=100, c='k')
ax2.scatter(plot_pixx2, plot_pixy2, marker='x', s=100, c='r')
ax2.set_title('Structure Image')
ax2.axis("off")

Generate training inputs (image patches) and targets (spectra)



In [ ]:
norm_ = lambda x: (x - x.min()) / x.ptp()

In [ ]:
img = norm_(struc_img)
spectra = norm_(pola_off_field)

coordinates = get_coord_grid(img, step = 1, return_dict=False)

# extract subimage for each point on a grid
window_size = 16
features_all, coords, _ = extract_subimages(img, coordinates, window_size)
features_all = features_all[:,:,:,0]
coords = np.array(coords, dtype=int)

print(coords.shape)
print(features_all.shape)
print(spectra.shape)

In [ ]:
#Plot an example patch and spectrum
k = 100

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(9, 3), dpi =200)
ax1.imshow(img)
ax1.scatter(coords[k, 1], coords[k, 0], marker='x', s=50, c='k')

ax1.axis("off")
ax2.imshow(features_all[k])
ax2.axis("off")
ax3.plot(v_step, spectra[coords[k, 0], coords[k, 1]])

In [ ]:
targets = spectra[coords[:, 0], coords[:, 1]]
print('target spectra shape: ', targets.shape)
print('image patch shape: ', features_all.shape)
# check if the spectra target is correct
plt.imshow(targets.reshape(img.shape[0]-window_size+1, img.shape[1]-window_size+1, -1).mean(2), origin = "lower")

Normalize data:

In [ ]:
features, targets = norm_(features_all), norm_(targets)

### Scalarizer
Next, we select a scalarizer function that will convert a measured spectrum into a scalar physical descriptor.

In [ ]:
def loop_area (raw_spec, cycle) :
  raw_spec_len = len(raw_spec)
  cycle_len = int(raw_spec_len / cycle)
  half_len = int(cycle_len / 2)
  q_len = int(cycle_len / 4)
  loop_top, loop_bottom = [], []
  loop_top.append(raw_spec[q_len : q_len + half_len])
  loop_top.append(raw_spec[q_len + 2*half_len : q_len + 3*half_len])
  loop_top.append(raw_spec[q_len + 4*half_len : 2*q_len + 4*half_len])
  loop_bottom.append(raw_spec[:q_len])
  loop_bottom.append(raw_spec[q_len + half_len: q_len + 2*half_len])
  loop_bottom.append(raw_spec[q_len + 3*half_len: q_len + 4*half_len])
  loop_top = np.concatenate(loop_top)
  loop_bottom = np.concatenate(loop_bottom)
  return np.abs(np.sum(loop_top)-np.sum(loop_bottom))

In [ ]:
# First way to extract scalarizer
loop_areas_all, features_all, indices_all = [], [], []
for i, t in enumerate(targets):
    looparea = loop_area(t, 3)
    loop_areas_all.append(np.array([looparea]))
    features_all.append(features[i])
    indices_all.append(coords[i])
loop_areas_all = np.concatenate(loop_areas_all)
features_all = np.array(features_all)
indices_all = np.array(indices_all)

# Plot the scalarized target value
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 4))
ax1.scatter(indices_all[:, 1], indices_all[:, 0], c=loop_areas_all)
ax1.set_title('Loop Area')
ax1.set_aspect('equal')

ax2.imshow(struc_img, origin = "lower")
ax2.set_title('Structure Image')
ax2.axis("off")

## Reconstruction from partial data
Here we demonstrate how to use DKL to learn a correlative structure-property relationship from a relatively small number of image-(scalarized)spectrum pairs and then use the trained model to predict a targeted physical property for the entire image space.

Prepare data:

In [ ]:
n, d1, d2 = features_all.shape
X = features_all.reshape(n, d1*d2)
y = norm_(loop_areas_all)
X.shape, y.shape

Split the data in such a way that we use only a relatively small part of data (to the left of the vertical dashed line in the figure below) to train a DKL model and then use the trained model to make a prediction of the "unmeasured" plasmon peak values (the part to the right of the vertical dashed line):

In [ ]:
split_ = 15
X_train = X[indices_all[:, 0] < split_]
y_train = y[indices_all[:, 0] < split_]
indices_train = indices_all[indices_all[:, 0] < split_]

_, ax = plt.subplots(dpi=100)
ax.scatter(indices_all[:, 1], indices_all[:, 0], s=50, c=y)
ax.hlines(split_, indices_all[:, 1].min(), indices_all[:, 1].max(), linestyle='--', color='w')
ax.set_title('Loop Area');
ax.set_aspect('equal')

Initialize and train a DKL model:

In [ ]:
data_dim = X_train.shape[-1]

key1, key2 = gpax.utils.get_keys()

dkl = gpax.viDKL(data_dim, z_dim=2, kernel='RBF')
dkl.fit(key1, X_train, y_train, num_steps=100, step_size=0.05)

Use the trained model to make a probabilsitic prediction for all the image patches:

Visualize predictive mean and uncertainty:

In [ ]:
mean, var = dkl.predict(key2, X)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4.5), dpi=100)
ax1.scatter(indices_all[:, 1], indices_all[:, 0], s=50, c=mean)
ax1.set_title("DKL prediction")
ax2.scatter(indices_all[:, 1], indices_all[:, 0], s=50, c=var)
ax2.set_title("DKL uncertainty")
for _ax in fig.axes:
    _ax.set_aspect('equal')

We can also visualize the latent/embedding space:

In [ ]:
embeded = dkl.embed(X)
embeded = embeded / embeded.max()

_, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
ax1.scatter(indices_all[:, 1], indices_all[:, 0], c=embeded[:, 0], cmap='RdBu')
ax2.scatter(indices_all[:, 1], indices_all[:, 0], c=embeded[:, 1], cmap='RdBu')

## Active learning
Now we are going to use DKL to actively learn (local) structures where a particular physical behavior or property (here, plasmon resonance peak) is maximized. Starting with just a few "measured" point, we use DKL to obtain predictive mean and variance for our property of interest over the entire parameter space, and then use them to compute the upper confedence bound (UCB) acquisition function for sampling the next measurement point.

Prepare the data

In [ ]:
n, d1, d2 = features_all.shape
X = features_all.reshape(n, d1*d2)
y = norm_(loop_areas_all)

exploration_steps = 200
testsize = 5

X.shape, y.shape


Get the initial measurements aka training points. Here ```X_measured``` are the already measured points, that is, the image patches for which there are measured spectra, whose scalarized values are stored in ```y_measured```. The ```X_unmeasured``` are unmeasured points, that is, image patches for which there are yet no measured spectra.

In [ ]:
# use only 0.02% of grid data points as initial training points
(X_measured, X_unmeasured, y_measured, y_unmeasured,
  indices_measured, indices_unmeasured) = train_test_split(
      X, y, indices_all, test_size=1-testsize/len(X), shuffle=True, random_state=1)

seed_points = len(X_measured)
seed_points

In [ ]:
plt.figure(figsize=(4, 4))
plt.imshow(struc_img, origin = "lower", cmap = "gray")
plt.scatter(indices_measured[:, 1], indices_measured[:, 0], s=50, c=y_measured, cmap = 'jet')

Do sample exploration based on the pre-acquired data (i.e., we are running a "dummy" experiment):

In [ ]:
def plot_result(indices, obj):
    plt.figure(figsize = (4, 4))
    plt.scatter(indices[:, 1], indices[:, 0], s=10, c=obj)
    next_point = indices[obj.argmax()]
    plt.scatter(next_point[1], next_point[0], s=10, marker='x', c='k')
    plt.xticks([])
    plt.yticks([])
    plt.title("Acquisition Values")
    plt.show()

In [ ]:
data_dim = X_measured.shape[-1]

key1, key2 = gpax.utils.get_keys()
for e in range(exploration_steps):
    print("{}/{}".format(e+1, exploration_steps))
    # update GP posterior
    dkl = gpax.viDKL(data_dim, 2)
    dkl.fit(  # you may decrease step size and increase number of steps (e.g. to 0.005 and 1000) for more stable performance
        key1, X_measured, y_measured, num_steps=200, step_size=0.01)
    # Compute UCB acquisition function
    obj = gpax.acquisition.UCB(key2, dkl, X_unmeasured)
    # Select next point to "measure"
    next_point_idx = obj.argmax()
    # Do "measurement"
    measured_point = y_unmeasured[next_point_idx]
    # Plot current result
    plot_result(indices_unmeasured, obj)
    np.savez("singleDKL_step{}.npz".format(e), next_point_idx = next_point_idx, obj = obj, X_measured = X_measured,
             X_unmeasured = X_unmeasured, y_measured = y_measured, y_unmeasured = y_unmeasured,
             indices_measured = indices_measured, indices_unmeasured = indices_unmeasured)
    # Update the arrays of measured/unmeasured points
    X_measured = np.append(X_measured, X_unmeasured[next_point_idx][None], 0)
    X_unmeasured = np.delete(X_unmeasured, next_point_idx, 0)
    y_measured = np.append(y_measured, measured_point)
    y_unmeasured = np.delete(y_unmeasured, next_point_idx)
    indices_measured = np.append(indices_measured, indices_unmeasured[next_point_idx][None], 0)
    indices_unmeasured = np.delete(indices_unmeasured, next_point_idx, 0)

In [ ]:
plt.figure(dpi = 200)
plt.imshow(img, origin="lower", cmap='gray')
plt.scatter(indices_measured[seed_points:, 1], indices_measured[seed_points:, 0],
            c=np.arange(len(indices_measured[seed_points:])), s=50, cmap="Reds")
plt.colorbar(label="Iteration")
plt.title("Single DKL Observations")
plt.xticks([])
plt.yticks([])

Overlay with a 'ground truth':

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(indices_all[:, 1], indices_all[:, 0], c=y, cmap='bwr', alpha=0.3)
plt.scatter(indices_measured[seed_points:, 1], indices_measured[seed_points:, 0],
            c=np.arange(len(indices_measured[seed_points:])), s=50, cmap="jet")
plt.colorbar()

In [ ]:
dkl1 = dkl
mean1, var1 = dkl.predict(key2, X)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4.5), dpi=200)
ax1.imshow(mean1.reshape((65,65)), origin = 'lower')
ax1.set_title("Prediction")
ax2.imshow(var1.reshape((65,65)), origin = 'lower')
ax2.set_title("Uncertainty")
for _ax in fig.axes:
    _ax.set_aspect('equal')
    _ax.set_xticks([])
    _ax.set_yticks([])

Add constrain

In [ ]:
img = spectra.mean(2)
coordinates = get_coord_grid(img, step = 1, return_dict=False)

# extract subimage for each point on a grid
features_all, coords, _ = extract_subimages(img, coordinates, window_size)
features_all = features_all[:,:,:,0]
coords = np.array(coords, dtype=int)
indices_all = np.array(coords, dtype=int)

print(coords.shape)
print(features_all.shape)
print(spectra.shape)

In [ ]:
#Plot an example patch and spectrum
k = 100; k1 = 2365; k2 = 3569

f, (ax1, ax3) = plt.subplots(1, 2, figsize=(8, 3), dpi =200)
ax1.imshow(img, origin = "lower")
ax1.set_title("Structure Image")
ax1.scatter(coords[k, 1], coords[k, 0], marker='x', s=50, c='black')
ax1.scatter(coords[k1, 1], coords[k1, 0], marker='x', s=50, c='red')
ax1.scatter(coords[k2, 1], coords[k2, 0], marker='x', s=50, c='orange')
ax1.set_xticks([])
ax1.set_yticks([])
ax3.plot(v_step, spectra[coords[k, 0], coords[k, 1]], c = 'black')
ax3.plot(v_step, spectra[coords[k1, 0], coords[k1, 1]], c = 'red')
ax3.plot(v_step, spectra[coords[k2, 0], coords[k2, 1]], c = 'orange')
ax3.set_xlabel("Voltage (V)")
ax3.set_ylabel("Piezoresponse (a.u.)")
ax3.set_title("Representative Spectra")

In [ ]:
ref_spec = spectra[coords[k, 0], coords[k, 1]]

In [ ]:
targets = spectra[coords[:, 0], coords[:, 1]]
print('target spectra shape: ', targets.shape)
print('image patch shape: ', features_all.shape)
#check if the spectra target is correct
plt.imshow(targets.reshape(img.shape[0]-window_size+1, img.shape[1]-window_size+1, -1).mean(2), origin = "lower")

In [ ]:
# Normalize data
features, targets = norm_(features_all), norm_(targets)

In [ ]:
# Test ssim
from skimage.metrics import structural_similarity as ssim
for i in range (30):
  req_spec = targets[i*100]
  plt.plot(v_step, norm_(ref_spec))
  plt.plot(v_step, norm_(req_spec))
  ss = ssim(norm_(ref_spec), norm_(req_spec), win_size = 21)
  print('SSIM = ', ss)
  plt.show()

In [ ]:
f, ax3 = plt.subplots(figsize=(4, 3.2), dpi =200)
ax3.plot(v_step, norm_(ref_spec), c = 'black', label = "Reference")
ax3.plot(v_step, norm_(targets[500]), c = 'red', label = "Target 1, Quality Score = 0.87")
ax3.plot(v_step, norm_(targets[2600]), c = 'orange', label = "Target 2, Quality Score = -0.11")
ax3.set_xlabel("Voltage (V)")
ax3.set_ylabel("Piezoresponse (a.u.)")
ax3.set_ylim([-0.05, 1.3])
ax3.legend(loc = 1)

In [ ]:
# Calculate all targets ssim

def cal_ssim (req_spec, ref_spec):
  ssims = []
  win_s = 21
  if len(req_spec.shape)==1:
    ss = ssim(norm_(ref_spec), norm_(req_spec), win_size = win_s)
    ssims.append(ss)
  else:
    for i in range (req_spec.shape[0]):
      ss = ssim(norm_(ref_spec), norm_(req_spec[i]), win_size = 21)
      ssims.append(ss)
  return np.asarray(ssims)

ssims = cal_ssim(targets, ref_spec)

#check the targets ssims
plt.imshow(ssims.reshape(img.shape[0]-window_size+1, img.shape[1]-window_size+1), origin = "lower")
plt.colorbar()

## Active learning
Now we are going to use DKL to actively learn (local) structures where a particular physical behavior or property (here, plasmon resonance peak) is maximized. Starting with just a few "measured" point, we use DKL to obtain predictive mean and variance for our property of interest over the entire parameter space, and then use them to compute the upper confedence bound (UCB) acquisition function for sampling the next measurement point.

In [ ]:
# Prepare the data
n, d1, d2 = features_all.shape
X = features_all.reshape(n, d1*d2)
y = norm_(loop_areas_all)
X.shape, y.shape

Get the initial measurements aka training points. Here ```X_measured``` are the already measured points, that is, the image patches for which there are measured spectra, whose scalarized values are stored in ```y_measured```. The ```X_unmeasured``` are unmeasured points, that is, image patches for which there are yet no measured spectra.

In [ ]:
testsize = 5
# use only 0.02% of grid data points as initial training points
(X_measured, X_unmeasured, y_measured, y_unmeasured, idx_train, idx_test,
  indices_measured, indices_unmeasured, targets_measured, targets_unmeasured) = train_test_split(
      X, y, np.arange(len(y)), indices_all, targets, test_size=1-testsize/len(X), shuffle=True, random_state=1)

seed_points = len(X_measured)
seed_points

In [ ]:
plt.figure(figsize=(4, 4))
plt.scatter(indices_measured[:, 1], indices_measured[:, 0], s=50, c=y_measured)

In [ ]:
# calculate ssim of seed targets
ssims_measured = cal_ssim(targets_measured, ref_spec)
plt.figure(figsize=(4, 4))
plt.scatter(indices_measured[:, 1], indices_measured[:, 0], s=50, c=ssims_measured)

In [ ]:
# Define a function to use DKL to predict constrains based on measured targets
def update_constrain(X_m, Constrain_measured, XALL, idx_all):
  k1, k2 = gpax.utils.get_keys()
  norm_(Constrain_measured)
  dkl_constrain = gpax.viDKL(data_dim, 2)
  dkl_constrain.fit(k1, X_m, Constrain_measured, num_steps=200, step_size=0.01)
  mean, var = dkl_constrain.predict(k2, XALL)
  plt.figure(figsize = (4,4), dpi = 200)
  plt.title("2nd GP Prediction")
  plt.scatter(idx_all[:,1], idx_all[:,0], s=10, c = mean, cmap = 'bwr')
  plt.xticks([])
  plt.yticks([])
  plt.show()

  return mean

In [ ]:
def interven (constrain_score, acq, X_train, y_train, indices_train,
              idx_train, idx_test, interven_percent=0.2):
  # Determine space, where the noise is above interven_percent level, to remove
  idx_for_filterout = np.where(constrain_score < np.quantile(constrain_score, interven_percent))[0]
  # Check if points in train set are from this space
  try:
    train_filterout_idx = np.where(np.isin(idx_train, idx_for_filterout))[0]
  except:
    train_filterout_idx = None
  # Remove points from train set
  if train_filterout_idx is not None:
    X_train_update = np.delete(X_train, train_filterout_idx, 0)
    y_train_update = np.delete(y_train, train_filterout_idx, 0)
    indices_train_update = np.delete(indices_train, train_filterout_idx, 0)
    idx_train_update = np.delete(idx_train, train_filterout_idx, 0)

  # Adjust acquisiton
  acq_idx_filterout = np.where(np.isin(idx_test, idx_for_filterout))[0]
  # 1st adjusting method
  adjust_acq_1 = acq.at[acq_idx_filterout].set(acq.min()) # set acq at removal space to 0
  # 2nd adjusting method
  constrain_score_testset = constrain_score[idx_test]
  adjust_acq_2 = (acq*constrain_score_testset).at[acq_idx_filterout].set(acq.min()) # tune acq with noise, then set tuned acq at removal space to 0

  return adjust_acq_1, adjust_acq_2, X_train_update, y_train_update, indices_train_update, idx_train_update

In [ ]:
def step_DKL(X_measured, y_measured, X_unmeasured):
  data_dim = X_measured.shape[-1]
  key1, key2 = gpax.utils.get_keys()
  # update GP posterior
  dkl = gpax.viDKL(data_dim, 2)
  dkl.fit(key1, X_measured, y_measured, num_steps=200, step_size=0.01)
  # Compute UCB acquisition function
  obj = gpax.acquisition.UCB(key2, dkl, X_unmeasured)
  return obj

In [ ]:
data_dim = X_measured.shape[-1]

for e in range(exploration_steps):
    print("{}/{}".format(e+1, exploration_steps))
    if e == 0:
      obj = step_DKL(X_measured, y_measured, X_unmeasured)
    else:
      obj = step_DKL(X_train_update, y_train_update, X_unmeasured)

    # Calculate constrain every .. step_size
    ssims_measured = cal_ssim(targets_measured, ref_spec)
    constrain = update_constrain(X_measured, ssims_measured, X, indices_all)
    (obj1, obj2, X_train_update, y_train_update,
     indices_train_update, idx_train_update) = interven(constrain, obj, X_measured,
                                                        y_measured, indices_measured, idx_train, idx_test)
    # update GP posterior

    print("Before Tune obj")
    plot_result(indices_unmeasured, obj)

    # Select next point to "measure"
    next_point_idx = obj1.argmax()
    # Do "measurement"
    measured_point = y_unmeasured[next_point_idx]
    # Plot current result
    print("After Tune obj")
    plot_result(indices_unmeasured, obj1)
    np.savez("DualDKL_step{}.npz".format(e), next_point_idx = next_point_idx, obj = obj, obj1 = obj1,
             X_measured = X_measured, constrain = constrain,
             X_unmeasured = X_unmeasured, y_measured = y_measured, y_unmeasured = y_unmeasured,
             indices_measured = indices_measured, indices_unmeasured = indices_unmeasured,
             idx_train = idx_train, idx_test = idx_test, targets_measured = targets_measured,
             targets_unmeasured = targets_unmeasured)
    # Update the arrays of measured/unmeasured points
    X_measured = np.append(X_measured, X_unmeasured[next_point_idx][None], 0)
    X_unmeasured = np.delete(X_unmeasured, next_point_idx, 0)
    y_measured = np.append(y_measured, measured_point)
    y_unmeasured = np.delete(y_unmeasured, next_point_idx)
    indices_measured = np.append(indices_measured, indices_unmeasured[next_point_idx][None], 0)
    indices_unmeasured = np.delete(indices_unmeasured, next_point_idx, 0)
    idx_train = np.append(idx_train, idx_test[next_point_idx][None], 0)
    idx_test = np.delete(idx_test, next_point_idx, 0)
    targets_measured = np.append(targets_measured, targets_unmeasured[next_point_idx][None], 0)
    targets_unmeasured = np.delete(targets_unmeasured, next_point_idx, 0)

In [ ]:
plt.figure(dpi = 200, figsize = (4, 4))
plt.imshow(img, origin="lower", cmap='gray')
plt.scatter(indices_measured[seed_points:, 1], indices_measured[seed_points:, 0],
            c=np.arange(len(indices_measured[seed_points:])), s=30, cmap="Reds")
plt.colorbar(shrink = 0.8, label="Iteration")
plt.title("Dual DKL Observations")
plt.xticks([])
plt.yticks([])

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(indices_all[:, 1], indices_all[:, 0], c=loop_areas_all, cmap='bwr', alpha=0.3)
plt.scatter(indices_measured[seed_points:, 1], indices_measured[seed_points:, 0],
            c=np.arange(len(indices_measured[seed_points:])), s=50, cmap="jet")
plt.colorbar()

In [ ]:
mean, var = dkl.predict(key2, X)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4.5), dpi=200)
ax1.imshow(mean.reshape((65,65)), origin = 'lower')
ax1.set_title("Prediction")
ax2.imshow(var.reshape((65,65)), origin = 'lower')
ax2.set_title("Uncertainty")
for _ax in fig.axes:
    _ax.set_aspect('equal')
    _ax.set_xticks([])
    _ax.set_yticks([])


# Data Analysis

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 4.5), dpi=200)
ax1.imshow(mean1.reshape((65,65)), origin = 'lower')
ax1.set_title("Single DKL")
ax2.imshow(mean.reshape((65,65)), origin = 'lower')
ax2.set_title("Dual DKL")
ax3.imshow(y.reshape((65,65)), origin = 'lower')
ax3.set_title("Ground Truth")
for _ax in fig.axes:
    _ax.set_aspect('equal')
    _ax.set_xticks([])
    _ax.set_yticks([])

In [ ]:
plt.figure(dpi = 200, figsize = (4, 4))
plt.imshow(norm_(mean1.reshape((65,65))), origin="lower")
plt.colorbar(shrink = 0.8, label="Normalized Loop Area (a.u.)")
plt.title("Single DKL")
plt.xticks([])
plt.yticks([])

In [ ]:
sdkl_final = np.load("singleDKL_step199.npz")
ddkl_final = np.load('DualDKL_step199.npz')

In [ ]:
d_y_measured = ddkl_final['y_measured']
s_y_measured = sdkl_final['y_measured']
st = np.arange(199)

fig, ax = plt.subplots(figsize=(4, 3.5), dpi=300)
ax.scatter(st, s_y_measured[5:], c = 'b', label = 'Single DKL', s = 15, alpha = 0.7)
ax.scatter(st, d_y_measured[5:], c = 'r', label = 'Dual DKL', s = 15, alpha = 0.7)
ax.set_xlabel("Iteration")
ax.set_ylabel("Normalized Loop Area")
ax.legend()

In [ ]:
s_indices_measured = sdkl_final['indices_measured']
d_indices_measured = ddkl_final['indices_measured']

In [ ]:
plt.figure(dpi = 200)
plt.imshow(img, origin="lower", cmap='gray')
plt.scatter(s_indices_measured[seed_points:, 1], s_indices_measured[seed_points:, 0],
            c=np.arange(len(s_indices_measured[seed_points:])), s=50, cmap="Reds")
plt.colorbar(label="Iteration")
plt.title("Single DKL Observations")
plt.xticks([])
plt.yticks([])

plt.figure(dpi = 200)
plt.imshow(img, origin="lower", cmap='gray')
plt.scatter(d_indices_measured[seed_points:, 1], d_indices_measured[seed_points:, 0],
            c=np.arange(len(d_indices_measured[seed_points:])), s=50, cmap="Reds")
plt.colorbar(label="Iteration")
plt.title("Dual DKL Observations")
plt.xticks([])
plt.yticks([])

In [ ]:
ddkl_step = np.load('DualDKL_step25.npz')
ddkl_step.files

In [ ]:
obj1 = ddkl_step['obj1']
obj = ddkl_step['obj']
indices_unmeasured = ddkl_step['indices_unmeasured']
constrain = ddkl_step['constrain']

In [ ]:
plt.figure(figsize = (5, 4), dpi = 200)
plt.scatter(indices_unmeasured[:, 1], indices_unmeasured[:, 0],
            c=norm_(obj), s=13, marker = "o")
plt.title("Single GP")
plt.colorbar(label = "Acquistion Value")
plt.xticks([])
plt.yticks([])

In [ ]:
plt.figure(figsize = (5, 4), dpi = 200)
plt.scatter(indices_unmeasured[:, 1], indices_unmeasured[:, 0],
            c=((obj1-np.nanmin(obj1))/np.nanmax(obj1))*1e10/1.92, s=13, marker = "o")
plt.title("Dual GP")
plt.colorbar(label = "Acquistion Value")
plt.xticks([])
plt.yticks([])

In [ ]:
plt.figure(figsize = (5, 4), dpi = 200)
plt.scatter(indices_all[:, 1], indices_all[:, 0],
            c=norm_(constrain), s=13, marker = "o")
plt.title("2nd GP Prediction")
plt.colorbar(label = "Quality Score")
plt.xticks([])
plt.yticks([])